In [55]:
import sys
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_extract, col

In [56]:
spark = SparkSession.builder \
    .appName("Spark Session") \
    .enableHiveSupport() \
    .getOrCreate()

In [57]:
spark.sql(f"CREATE DATABASE IF NOT EXISTS silver")

DataFrame[]

In [58]:
df = spark.sql("REFRESH TABLE bronze.nubank_extrato")
df = spark.sql("REFRESH TABLE bronze.nubank_fatura")

In [59]:
df = spark.sql("""
SELECT 
DISTINCT
Identificador, 
cast(to_date(from_unixtime(unix_timestamp(Data, 'dd/MM/yyyy'))) as date) AS Data,
cast(to_date(from_unixtime(unix_timestamp(Data, 'dd/MM/yyyy'))) as date) AS Data_Pgto,
Descricao,
'Debito' AS DebitoCredito,
CASE WHEN Valor < 0 THEN 'PAGO' ELSE 'RECEBIDO' END AS Tipo,
Valor
FROM bronze.nubank_extrato
""")

In [60]:
pattern = r"(?<= - )([^-\d]+)"

df = df.withColumn("Entidade", regexp_extract(col("Descricao"), pattern, 1))

In [61]:
df.coalesce(1).write.mode("overwrite").format("parquet").saveAsTable(f"silver.nubank_extrato")

In [62]:
df = spark.sql("""
SELECT
DISTINCT
date AS Data,
CAST(SUBSTRING(arquivo_origem,8,10) AS date) AS Data_Pgto,
title AS Descricao,
'Credito' AS DebitoCredito,
'PAGO' AS Tipo,
(amount * -1) AS Valor
FROM bronze.nubank_fatura
""")

In [63]:
pattern = r"^(.*?)(?: - |$)"

df = df.withColumn("Entidade", regexp_extract(col("Descricao"), pattern, 1))

In [64]:
df.coalesce(1).write.mode("overwrite").format("parquet").saveAsTable(f"silver.nubank_fatura")